<a href="https://colab.research.google.com/github/DuyguA/data_science_scripts/blob/master/xlmbert_autoencoder_diger_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo pip install transformers

     |████████████████████████████████| 1.3MB 5.7MB/s 
     |████████████████████████████████| 2.9MB 38.1MB/s 
     |████████████████████████████████| 1.1MB 50.0MB/s 
     |████████████████████████████████| 890kB 56.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=8b5f67b41c9f6f855c76f15a124860c8a3c414e94c75f943d8aa9c5858daf90b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

2.3.0


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Bidirectional, Dropout, Input, Embedding, GlobalMaxPool1D, TimeDistributed, RepeatVector, Lambda
from keras import optimizers
import numpy as np


In [ ]:
sentences = []

lemmas = []
analysis = []

with open("turk.txt", "r", encoding="utf-8") as file:
    for line in file:
        l = line.strip()
        word, res = l.split(" ")
        word = word.strip()
        lemma, ana = res.strip().split(",", 1)
        analysis.append(ana.split(","))
        sentences.append(word)
        lemmas.append(lemma)

In [ ]:
sentences=sentences[:1000]
lemmas = lemmas[:1000]
analysis = analysis[:1000]

In [ ]:
tokenizer = Tokenizer(char_level=True, filters=None, lower=True)

tokenizer.fit_on_texts(lemmas)

lemma_seqs = tokenizer.texts_to_sequences(lemmas)

In [ ]:
OUT_LEN = 20

lemma_char = pad_sequences(lemma_seqs, OUT_LEN, padding="post")

lemmas = np.array(lemma_char)

lemmas = lemmas.reshape(len(lemmas) , OUT_LEN, 1)


In [ ]:
atokenizer = Tokenizer(lower=True)
atokenizer.fit_on_texts(analysis)
anl_seqs = atokenizer.texts_to_sequences(analysis) 

In [ ]:
anl_word = pad_sequences(anl_seqs, 10, padding="post")
analysis = np.array(anl_word)
analysis = analysis.reshape(1000 , 10, 1)

In [ ]:
w =[xtokenizer.encode(x, add_special_tokens=True, max_length=30, pad_to_max_length=True) for x in sentences]


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
xtokenizer.encode("Hello!How are you madam?", add_special_tokens=True, max_length=30, padding=True, return_tensors="tf")

<tf.Tensor: shape=(1, 10), dtype=int32, numpy=
array([[    0, 50944,   267, 18719,    95,    79,  5111,   209,   112,
            1]], dtype=int32)>

In [ ]:
from transformers import XLMTokenizer, TFXLMForSequenceClassification, TFXLMModel

xtokenizer = XLMTokenizer.from_pretrained('xlm-mlm-xnli15-1024')
#xmodel = TFXLMForSequenceClassification.from_pretrained('xlm-mlm-17-1280')
xmodel = TFXLMModel.from_pretrained("xlm-mlm-xnli15-1024", use_lang_emb=True)

Some layers from the model checkpoint at xlm-mlm-xnli15-1024 were not used when initializing TFXLMModel: ['pred_layer_._proj']
- This IS expected if you are initializing TFXLMModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLMModel were initialized from the model checkpoint at xlm-mlm-xnli15-1024.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMModel for predictions without further training.


In [ ]:
w= np.array(w)

numpy.ndarray

In [ ]:
input_layer = Input(shape = (30,), dtype='int32')
bert = xmodel(input_layer)
bert = bert[0]

lstm = LSTM(units=100, return_sequences=False)(bert)  # variational biLSTM
rv = RepeatVector(OUT_LEN)(lstm)
lstmd = Bidirectional(LSTM(units=100, return_sequences=True))(rv)
outputs = TimeDistributed(Dense(30, activation="softmax"))(lstmd)

model = Model(inputs=input_layer, outputs=outputs)

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["sparse_categorical_accuracy"])

model.summary()

history = model.fit(w, lemmas, batch_size=32, epochs=5, verbose=1 )#, callbacks=[tensorboard_callback]), validation_split=0.1)



Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
tfxlm_model (TFXLMModel)     ((None, 30, 1024),)       248976384 
_________________________________________________________________
lstm (LSTM)                  (None, 100)               450000    
_________________________________________________________________
repeat_vector (RepeatVector) (None, 20, 100)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 20, 200)           160800    
_________________________________________________________________
time_distributed (TimeDistri (None, 20, 30)            6030      
Total params: 249,593,214
Trainable params: 249,593,214
Non-trainable params: 0
________________________________________

In [ ]:
input_layer = Input(shape = (30,), dtype='int32')
bert = xmodel(input_layer)
bert = bert[0]

lstm = LSTM(units=100, return_sequences=False)(bert)  # variational biLSTM
rv = RepeatVector(30)(lstm)
lstmd = LSTM(units=100, return_sequences=True)(rv)

lemma_output = Lambda(lambda x: x[:,:20,] )(lstmd)
analysis_output = Lambda(lambda x: x[:,20:,:])(lstmd)

lemma_output = TimeDistributed(Dense(30, activation="softmax"))(lemma_output)
analysis_output = TimeDistributed(Dense(66, activation="softmax"))(analysis_output)


model = Model(inputs=input_layer, outputs=[lemma_output, analysis_output])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["sparse_categorical_accuracy"])

model.summary()

history = model.fit(w, [lemmas, analysis], batch_size=32, epochs=5, verbose=1 )#, callbacks=[tensorboard_callback]), validation_split=0.1)



Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 30)]         0                                            
__________________________________________________________________________________________________
tfxlm_model (TFXLMModel)        ((None, 30, 1024),)  248976384   input_3[0][0]                    
__________________________________________________________________________________________________
lstm_4 (LSTM)                   (None, 100)          450000      tfxlm_model[2][0]                
__________________________________________________________________________________________________
repeat_vector_2 (RepeatVector)  (None, 30, 100)      0           lstm_4[0][0]                     
_______________________________________________________________________________________